In [102]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [103]:
df= pd.read_csv("base_vin_v1.csv")
df.head()


C:\Users\116_9\AppData\Local\Temp\ipykernel_12612\2265193916.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("base_vin_v1.csv")


,Unnamed: 0,nom,desc,accroche,visuel,producteur,contenance,deg_alcool,pays,region,...,type_effervescent,type_vin,couleur_blanc,couleur_rouge,couleur_rose,couleur_orange,couleur_ambre,couleur_rubis,prix_std,degres_std
0,0,Colombard-Sauvignon 2024 - Horgelus,L'harmonieux mariage du colombard et du sauvig...,Savoureux mariage du colombard et du sauvignon...,https://www.vinatis.com/72739-detail_default/c...,Horgelus,0.75,11.5,France,Sud-Ouest,...,0,1,1,0,0,0,0,0,-0.117300,-1.658776
1,1,Le Bal des Papillons 2024 - Horgelus,Élaborée avec soin dans le respect des traditi...,Plongez dans un univers de légèreté et délégan...,https://www.vinatis.com/73560-detail_default/l...,Horgelus,0.75,11.0,France,Sud-Ouest,...,0,1,1,0,0,0,0,0,-0.113351,-2.117774
2,2,Villa des Anges - Réserve 2021 - Jeff Carrel,La gamme Villa des Anges fait carton plein dep...,Un cuvée emblématique de la sélection qui ravi...,https://www.vinatis.com/57419-detail_default/v...,Jeff Carrel,0.75,14.0,France,NaN,...,0,1,0,1,0,0,0,0,-0.111574,0.636215
3,3,Lupo Meraviglia Tre di Tre 2022 - Botter,"Historiquement, la vinification dans les Pouil...",Une étoile montante des Pouilles vinifiée selo...,https://www.vinatis.com/63000-detail_default/l...,Botter,0.75,14.5,Italie,Pouilles,...,0,1,0,1,0,0,0,0,-0.108810,1.095213
4,4,N°3 - Colombard-Sauvignon Blanc 2024 - UBY,Léger et fruité ! Ce vin blanc aux arômes d'ag...,La référence incontestable des Côtes de Gascog...,https://www.vinatis.com/75247-detail_default/n...,UBY,0.75,11.0,France,Sud-Ouest,...,0,1,1,0,0,0,0,0,-0.117497,-2.117774


In [104]:
### Systeme de recommandation (non supervisé)

In [105]:
colonnes_objet = df.select_dtypes(include='object').columns
print(colonnes_objet)  # Pour voir la liste

Index(['nom', 'desc', 'accroche', 'visuel', 'producteur', 'pays', 'region',
       'appellation', 'cepages', 'millesime', 'accords', 'gout', 'caractere',
       'couleur', 'temp_serv', 'type_produit', 'bio', 'cepages_trouves'],
      dtype='object')


In [106]:
# 1. Assure-toi que 'bio_bool' existe
if 'bio_bool' not in df.columns:
    df['bio_bool'] = df['bio'].apply(lambda x: 1 if isinstance(x, str) and 'bio' in x.lower() else 0)

je garde dans df_num:

Toutes les colonnes numériques sauf deg_alcool,contennance  et prix,

Le nom du vin (nom),

Supprimer toutes les autres colonnes objet (texte, etc).



In [107]:
# 1. Colonnes objet
colonnes_objet = df.select_dtypes(include='object').columns

# 2. Conversion explicite en liste
colonnes_objet = list(colonnes_objet)

# 3. Colonnes numériques à supprimer
colonnes_num_a_supprimer = ['deg_alcool', 'prix', 'contenance']

# 4. On ajoute 'nom' à supprimer
colonnes_a_supprimer = colonnes_objet + colonnes_num_a_supprimer

# 5. Suppression dans le DataFrame (en ignorant les colonnes qui n'existent pas avec errors='ignore')
df_num = df.drop(columns=colonnes_a_supprimer, errors='ignore')

# 6. Vérification
print(df_num.info())
print(df_num.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4431 entries, 0 to 4430
Columns: 187 entries, Unnamed: 0 to bio_bool
dtypes: float64(2), int64(185)
memory usage: 6.3 MB
None
   Unnamed: 0  accord_aperitif  accord_barbecue  accord_champignon  \
0           0                1                0                  0   
1           1                1                0                  0   
2           2                0                1                  1   
3           3                1                1                  1   
4           4                1                0                  0   

   accord_charcuterie  accord_charcuterie_corse  \
0                   0                         0   
1                   0                         0   
2                   1                         0   
3                   1                         0   
4                   0                         0   

   accord_coquillage_et_crustace  accord_cuisine_du_monde  \
0                              0   

In [108]:
df_num.isna().sum()


Unnamed: 0            0
accord_aperitif       0
accord_barbecue       0
accord_champignon     0
accord_charcuterie    0
                     ..
couleur_ambre         0
couleur_rubis         0
prix_std              0
degres_std            1
bio_bool              0
Length: 187, dtype: int64

In [109]:
# Afficher toutes les lignes où degres_std est NaN
print(df[df['degres_std'].isna()])


      Unnamed: 0                                                nom desc  \
2204        2499  Pack 1 Bouteille Chandon Garden Spritz + Ice B...  NaN   

     accroche                                             visuel producteur  \
2204      NaN  https://www.vinatis.com/70091-detail_default/p...    Chandon   

      contenance  deg_alcool       pays   region  ... type_vin couleur_blanc  \
2204        0.75         NaN  Argentine  Mendoza  ...        0             1   

     couleur_rouge couleur_rose couleur_orange couleur_ambre couleur_rubis  \
2204             0            0              0             0             0   

      prix_std degres_std bio_bool  
2204 -0.091041        NaN        0  

[1 rows x 208 columns]


In [110]:

#suppression de la ligne na degre_std

df_num = df_num.dropna(subset=['degres_std'])


In [111]:
from sklearn.neighbors import NearestNeighbors
# On déclare notre algorithme des plus proches voisins et on lui 
# déclare combien de recommendations nous voulons récupérer (ici 6)
neigh = NearestNeighbors(n_neighbors=6)
# On entraine notre modèle sur les données normalisées (qui sont
# pertinentes pour notre algo de recommendation, numériques et 
# catégorielles)
neigh.fit(df_num)

NearestNeighbors(n_neighbors=6)

In [112]:
vin_cible = "Villa des Anges - Réserve 2021 - Jeff Carrel"  # ← Remplace par le nom du vin à recommander
# Cherche son index dans df
index_cible = df[df['nom'] == vin_cible].index[0]
# On récupère le vecteur des features du vin cible
vecteur_cible = df_num.iloc[index_cible].values.reshape(1, -1)

# Recherche des voisins
n_voisins = 10  # on prend large pour avoir plus de choix
distances, indices_voisins = neigh.kneighbors(vecteur_cible, n_neighbors=n_voisins)
indices_voisins = indices_voisins[0]
distances = distances[0]


c:\Users\116_9\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [113]:
# DataFrame des voisins, exclusion du vin cible
indices_voisins_sans_cible = [i for i in indices_voisins if i != index_cible]
df_voisins = df.iloc[indices_voisins_sans_cible].copy()
df_voisins['distance'] = [distances[j] for j, idx in enumerate(indices_voisins) if idx != index_cible]

# On prend les 4 plus proches (sans distinction)
df_reco_non_bio = df_voisins.head(4)

# On cherche le vin bio le plus proche de tout le DataFrame, hors les déjà sélectionnés
df_bio = df[(df['bio_bool'] == 1) & (~df.index.isin(df_reco_non_bio.index))]
if not df_bio.empty:
    # On calcule la distance de chaque vin bio au vin cible (en utilisant les features numériques)
    distances_bio = neigh.kneighbors(df_num.loc[df_bio.index], n_neighbors=1, return_distance=True)[0][:, 0]
    # Ajoute la distance au DataFrame
    df_bio = df_bio.copy()
    df_bio['distance'] = distances_bio
    # Prend le plus proche
    bio_suppl = df_bio.sort_values('distance').head(1)
    # Ajoute à la liste finale
    df_reco = pd.concat([df_reco_non_bio, bio_suppl])
else:
    df_reco = df_reco_non_bio  # S'il n'y a aucun bio dans toute la base

print(df_reco[['nom', 'bio_bool', 'distance','visuel']])

                                           nom  bio_bool  distance  \
3     Lupo Meraviglia Tre di Tre 2022 - Botter         0  2.282693   
1         Le Bal des Papillons 2024 - Horgelus         0  4.752311   
0          Colombard-Sauvignon 2024 - Horgelus         0  4.823589   
4   N°3 - Colombard-Sauvignon Blanc 2024 - UBY         0  4.958275   
48       Rosé 2024 - Domaine la Rose des Vents         1  0.000000   

                                               visuel  
3   https://www.vinatis.com/63000-detail_default/l...  
1   https://www.vinatis.com/73560-detail_default/l...  
0   https://www.vinatis.com/72739-detail_default/c...  
4   https://www.vinatis.com/75247-detail_default/n...  
48  https://www.vinatis.com/73079-detail_default/r...  
